# Лабораторная работа №2 (Подготовка данных в Pandas. Часть 2)

### 1. Создать новую переменную Total_count, в которую записать общее количество арендованных велосипедов за каждый час.

In [ ]:
import pandas as pd
import datetime
import numpy as np

df = pd.read_csv("BikeData.csv")
df['Total_count'] = df['Partner 1'] + df['Partner 2']
display(df)

### 2. Удалить ряды Partner 1 и Partner 2

In [ ]:
df = df.drop(columns=['Partner 1', 'Partner 2'])
display(df)

### 3. Преобразовать Date к стандартной форме

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst = True)
display(df)

### 4. Определить сколько велосипедов было выдано в день выполнения задания, определить день недели

In [ ]:
today = datetime.datetime.today()
filtered_df = df[(df['Date'].dt.day == today.day) & (df['Date'].dt.month == today.month)]
#display(filtered_df)
total_bikes_rented = filtered_df['Total_count'].sum()
day_of_week = filtered_df['Date'].iloc[0].strftime('%A')
print(f"Всего велосипедов продано: {total_bikes_rented}")
print(f"День недели: {day_of_week}")

### 5. Перекодировка Functioning Day в boolean

In [ ]:
df['Functioning Day'] = df['Functioning Day'].map({'Yes': True, 'No': False})
display(df)

#### 5.1. Подсчет нерабочих дней

In [ ]:
daily_hours = df.groupby(df['Date'].dt.date)['Functioning Day'].sum()
non_working_days = (daily_hours < 12).sum()
print(f"Число нерабочих дней: {non_working_days}")

### 6. Перекодировка Holiday в 0 и 1

In [ ]:
df['Holiday'] = df['Holiday'].apply(lambda x: 1 if x == 'Holiday' else 0)
display(df)

### 7. Ввод новой категориальной переменной Temperature category

In [ ]:
df['Temperature category'] = df['Temperature'].apply(lambda x: 'Freezing' if x < 0 else 'Chilly' if 0 <= x < 15 else 'Nice' if 15 <= x < 26 else 'Hot')
display(df)

### 8. Создать новую переменную Good weather

In [ ]:
df['Good weather'] = ((df['Temperature category'] == 'Nice') & 
                      (df['Humidity'] >= 40) & (df['Humidity'] <= 60) &
                      (df['Wind speed'] < 5.4) &
                      (df['Rainfall'] == 0) &
                      (df['Snowfall'] == 0)
                      ).astype(int)
display(df)
good_weather_pct = df['Good weather'].mean() * 100
print(f"Процент дней с хорошей погодой: {good_weather_pct:.2f}%")

### 9. Сгруппировать данные по сезонам (Seasons), внутри сезона по категории погоды (Temperature category) и вывести для полученной группировки количество арендованных велосипедов.

#### 9.1. Группировка через group by

In [ ]:
grouped = df.groupby(['Seasons', 'Temperature category'])['Total_count'].sum()
display(grouped)

#### 9.2. Групировка через сводную таблицу

In [ ]:
pivot = df.pivot_table(index='Seasons', columns='Temperature category', values='Total_count', aggfunc='sum')
display(pivot)

### 10. Определить сезон и температуру при которых выдается наибольшее и наименьшее количество велосипедов

In [47]:
max_row = grouped.idxmax()
min_row = grouped.idxmin()
print(f"Максимальное количество велосипедов арендовано в сезоне {max_row[0]} при температуре {max_row[1]}, количество велосипедов: {grouped[max_row]}")
print(f"Минимальное количество велосипедов арендовано в сезоне {min_row[0]} при температуре {min_row[1]}, количество велосипедов: {grouped[min_row]}")

Максимальное количество велосипедов арендовано в сезоне Summer при температуре Hot, количество велосипедов: 1347262
Минимальное количество велосипедов арендовано в сезоне Spring при температуре Freezing, количество велосипедов: 6331
